In [ ]:
import pandas as pd
import math
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import VarianceThreshold, mutual_info_classif
from datetime import datetime, timedelta, timezone
import pyodbc
from sqlalchemy import Column, Date, Integer, String, Numeric, create_engine, Float, inspect, func, MetaData, Table, select, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import mapper, registry, Session, relationship
from sqlalchemy.ext.automap import automap_base
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, mean_squared_error, balanced_accuracy_score 
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.base import BaseEstimator
from sklearn.pipeline import Pipeline
import scipy
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [ ]:
#SQLAlchemy Setup
def get_engine():
    SERVER = 'localhost'
    DATABASE = 'metmast'
    engine = create_engine(f'mssql+pyodbc:///?odbc_connect=DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={SERVER};DATABASE={DATABASE};Trusted_Connection=yes;TrustServerCertificate=yes;')
    return engine

engine = get_engine()
metadata = MetaData()
metadata.reflect(bind=engine)
Base = automap_base(metadata=metadata)
Base.prepare(autoload_with=engine)
mapper_registry = registry()

In [ ]:
datapoints = pd.read_sql_table("datapoint", engine)
feature_values_int = pd.read_sql_table("datapoint_feature_value_int", engine)
feature_values_float = pd.read_sql_table("datapoint_feature_value_float", engine)
features = pd.read_sql_table("feature", engine)
features = features[features["type_"]=="data"]
feature_list = []
feature_dict = dict()
feature_value_array_dict = dict()

for index, row in DataFrame(features).iterrows():
    feature = row["name"]
    index = row["id"]
    feature_dict[index] = feature
    feature_list.append(feature)
    feature_value_array_dict[feature] = np.empty(datapoints.shape[0], dtype="float")    
datapoints = pd.concat([datapoints, pd.DataFrame(columns=feature_list)], axis=1)

for index, row in DataFrame(feature_values_int).iterrows():
    feature = feature_dict[row["feature_id"]]
    row_id = row["datapoint_id"] - 1
    feature_value_array_dict[feature][row_id] = row["int_"]
for index, row in DataFrame(feature_values_float).iterrows():
    feature = feature_dict[int(row["feature_id"])]
    row_id = int(row["datapoint_id"]) - 1
    feature_value_array_dict[feature][row_id] = row["float_"]
    
    
for feature in feature_list:
    datapoints[feature] = feature_value_array_dict[feature]
train = datapoints[datapoints['datapoint_mappings_id'] == 1]
train.drop(columns=["datapoint_mappings_id"])
test = datapoints[datapoints['datapoint_mappings_id'] == 2]
test.drop(columns=["datapoint_mappings_id"])
print("Data loaded.")

In [ ]:
def update_values(obj, ref_class, session, **kwargs):
    #clone = ref_class()
    for column in ref_class.__table__.columns:
        if(column.name in kwargs):
            setattr(obj, column.name, kwargs[column.name])
        #else:
            #setattr(clone, column.name, getattr(obj, column.name))
    #session.delete(obj)
    #session.add(clone)
    session.commit() 
    return obj  

#Lazy declarations
def get_table_by_name(name):
    inspector = inspect(engine)
    return inspector.get_table(name)

def get_feature_id_by_name(Base, name):
    query = session.query(metadata.tables["feature"])
    df = pd.DataFrame()
    for batch in pd.read_sql_query(query.statement, engine, chunksize= 5):
        df = pd.concat([df, batch], ignore_index=True)
    for index, row in df.iterrows():
        if(row["name"]==name):
            return row["id"]
        
def getNextIdForTable(Base, name):
    table = Base.classes[name]
    return session.query(table.id).count() + 1 
  
def create_object(Base, class_name, **kwargs):
    mapped_class = Base.classes[class_name]
    # Create an instance of the mapped class
    obj = mapped_class()
    # Set the object's attributes
    for key, value in kwargs.items():
        setattr(obj, key, value)
    return obj

In [ ]:
session = Session(engine)
#Get table refrences
train_process_table = Base.classes["train_process"]
train_process_init_parameter_table = Base.classes["train_process_init_parameter"]
#Load initial parameters - use latest parameters if none were explicitly given
count_process = session.query(Base.classes.train_process.id).count()
count_paras = session.query(Base.classes.train_process_init_parameter.id).count()
init_parameters = session.query(train_process_init_parameter_table).order_by(train_process_init_parameter_table.id.desc()).limit(1)[0]
if(count_process == count_paras):    
    init_parameters_clone = Base.classes.train_process_init_parameter()
    for column in Base.classes.train_process_init_parameter.__table__.columns:
        if(column.name!="id"):
            setattr(init_parameters_clone, column.name, getattr(init_parameters, column.name))
        else:
            setattr(init_parameters_clone, column.name, getattr(init_parameters, column.name ) + 1)
    session.add(init_parameters_clone)
    session.commit()
#Create meta db Entrys
train_process = create_object(Base, "train_process", id = count_process + 1)
session.add(train_process)
session.commit()
init_parameters = update_values(init_parameters, Base.classes["train_process_init_parameter"], session, train_process_id = train_process.id )